# Bitcoin price prediction

## Imports

In [18]:
#Packages
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from pandas_datareader import data
from prophet.plot import plot_plotly, plot_components


from prophet import Prophet


In [2]:
# Bitcoin dataset
btc_data = data.DataReader("BTC-USD",
                          start='2013-04-28',
                          data_source='yahoo')['Adj Close']

btc = pd.DataFrame(btc_data)


In [16]:
btc.head()

,Adj Close
Date,
2014-09-17,457.334015
2014-09-18,424.440002
2014-09-19,394.795990
2014-09-20,408.903992
2014-09-21,398.821014


In [19]:
# Visualitation 
px.area(btc,
        x=btc.index,
        y='Adj Close')

In [4]:
# Taking the logs
btc['log_price'] = np.log(btc["Adj Close"])

px.area(btc,
        x=btc.index,
        y='log_price')


In [8]:
# Renaming the columns for Prophet
prophet_df = pd.DataFrame(btc['log_price'])
prophet_df['ds'] = btc.index
prophet_df.rename(columns={'log_price':'y'}, inplace=True)
prophet_df.head()

,y,ds
Date,,
2014-09-17,6.125414,2014-09-17
2014-09-18,6.050771,2014-09-18
2014-09-19,5.978369,2014-09-19
2014-09-20,6.013480,2014-09-20
2014-09-21,5.988513,2014-09-21


In [10]:
import itertools
import numpy as np
import pandas as pd
from prophet.diagnostics import cross_validation, performance_metrics


param_grid= {
    'changepoint_prior_scale':[0.001, 0.01, 0.1, 10], 
    'seasonality_prior_scale':[0.01, 0.1, 1.0, 5],
}

all_params= [dict(zip(param_grid.keys(), values))
             for values in itertools.product(*param_grid.values())]
rmses= []

for params in all_params:
    m= Prophet(**params)
    m.add_country_holidays(country_name='US')
    m.fit(prophet_df)
    df_cv = cross_validation(m, initial='1095 days', period='180 days', horizon = '365 days')
    df_p= performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

tuning_results= pd.DataFrame(all_params)
tuning_results['rmses']= rmses


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2017-09-23 00:00:00 and 2021-03-06 00:00:00
100%|██████████| 8/8 [00:42<00:00,  5.26s/it]
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2017-09-23 00:00:00 and 2021-03-06 00:00:00
100%|██████████| 8/8 [00:42<00:00,  5.34s/it]
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2017-09-23 00:00:00 and 2021-03-06 00:00:00
100%|██████████| 8/8 [00:41<00:00,  5.17s/it]
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2017-09-23 00:00:00 and 2021-03-06 00:00:00
100%|██████████| 8/8 [00:39<00:00,  4.89s/it]
INFO:prophet:Disabling daily

In [14]:
tuning_results.sort_values(by=['rmses'])

,changepoint_prior_scale,seasonality_prior_scale,rmses
9,0.100,0.10,1.049181
11,0.100,5.00,1.056197
10,0.100,1.00,1.066921
8,0.100,0.01,1.079062
2,0.001,1.00,1.167829
3,0.001,5.00,1.172395
0,0.001,0.01,1.173130
1,0.001,0.10,1.178447
12,10.000,0.01,1.184549
6,0.010,1.00,1.219802


In [15]:
model = Prophet(changepoint_prior_scale=0.1, seasonality_prior_scale=0.1)
model.add_country_holidays(country_name='US')
model.fit(prophet_df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [17]:
future= model.make_future_dataframe(61)
forecast = model.predict(future)

In [19]:
plot_plotly(model, forecast)

In [22]:
# Bitcoin dataset
SP  = data.DataReader("^GSPC",
                          start='2013-04-28',
                          data_source='yahoo')['Adj Close']

market = pd.DataFrame(SP)

In [31]:
prophet_df['market'] = np.log(market['Adj Close'])
prophet_df.dropna(axis=0, inplace=True)

In [32]:
model2 = Prophet(changepoint_prior_scale=0.1, seasonality_prior_scale=0.1)
model2.add_regressor('market')
model2.add_country_holidays(country_name='US')
model2.fit(prophet_df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [34]:
forecast = model2.predict(prophet_df)

In [35]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Christmas Day,Christmas Day_lower,Christmas Day_upper,Christmas Day (Observed),...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2014-09-17,7.160680,5.850595,6.294807,7.160680,7.160680,0.0,0.0,0.0,0.0,...,0.000411,0.000411,0.000411,-0.222444,-0.222444,-0.222444,0.0,0.0,0.0,6.076826
1,2014-09-18,7.154258,5.861272,6.294821,7.154258,7.154258,0.0,0.0,0.0,0.0,...,-0.001623,-0.001623,-0.001623,-0.225888,-0.225888,-0.225888,0.0,0.0,0.0,6.078414
2,2014-09-19,7.147836,5.858290,6.286591,7.147836,7.147836,0.0,0.0,0.0,0.0,...,0.000587,0.000587,0.000587,-0.229067,-0.229067,-0.229067,0.0,0.0,0.0,6.069702
3,2014-09-22,7.128569,5.816864,6.229268,7.128569,7.128569,0.0,0.0,0.0,0.0,...,0.001121,0.001121,0.001121,-0.236649,-0.236649,-0.236649,0.0,0.0,0.0,6.021147
4,2014-09-23,7.122147,5.785959,6.214686,7.122147,7.122147,0.0,0.0,0.0,0.0,...,-0.000062,-0.000062,-0.000062,-0.238412,-0.238412,-0.238412,0.0,0.0,0.0,5.995764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874,2022-02-25,9.554001,10.776676,11.218034,9.554001,9.554001,0.0,0.0,0.0,0.0,...,0.000587,0.000587,0.000587,0.142354,0.142354,0.142354,0.0,0.0,0.0,11.002882
1875,2022-02-28,9.556251,10.803690,11.227516,9.556251,9.556251,0.0,0.0,0.0,0.0,...,0.001121,0.001121,0.001121,0.160619,0.160619,0.160619,0.0,0.0,0.0,11.017171
1876,2022-03-01,9.557001,10.770069,11.180916,9.557001,9.557001,0.0,0.0,0.0,0.0,...,-0.000062,-0.000062,-0.000062,0.166038,0.166038,0.166038,0.0,0.0,0.0,10.979047
1877,2022-03-02,9.557751,10.816567,11.241813,9.557751,9.557751,0.0,0.0,0.0,0.0,...,0.000411,0.000411,0.000411,0.171106,0.171106,0.171106,0.0,0.0,0.0,11.036399


In [ ]:
plot_plotly(model2, forecast)